In [ ]:
pip install pypdf sentence_transformers qdrant_client

In [2]:
from langchain.document_loaders import PyPDFLoader 
from langchain.text_splitter import CharacterTextSplitter 
import re  

# 將一份 PDF 切成多個 chunks 
# 用 \n 分割, 每個 chunk size 上限為 256
def pdf_to_chunk(path, start_page=1, end_page=None): 
    # 使用 PyPDFLoader 來加載 PDF 文件
    loader = PyPDFLoader(path) 

    # 將 PDF 文件的每一頁作為單獨的對象加載到 pages 列表中
    pages = loader.load() 

    # 計算 PDF 文件的總頁數
    total_pages = len(pages) 
    
    # 創建一個 CharacterTextSplitter 對象
    # 用來將文本分割成多個塊
    text_splitter = CharacterTextSplitter( 
        separator="\\n",  # 使用 "\n" 作為分隔符
        chunk_size=256,  # 每個塊的最大字符數為 256
        chunk_overlap=20  # 塊之間有 20 個字符的重疊
    ) 

    # 如果沒有指定結束頁面，則默認處理到文檔的最後一頁
    if end_page is None: 
        end_page = len(pages) 

    # 用來存儲所有塊的列表
    lst_text = [] 
    
    # 循環處理指定頁面範圍內的每一頁
    for i in range(start_page-1, end_page): 
        # 將當前頁面的文本分割成多個塊
        chuncks = text_splitter.split_text(pages[i].page_content) 

        # 遍歷每一個塊
        for chunk in chuncks: 
            # 使用正則表達式去除多餘的空白字符
            text = re.sub(r'\\s+', ' ', chunk) 

            # 將處理後的文本塊添加到列表中
            lst_text.append(text) 
    
    # 返回包含所有文本塊的列表
    return lst_text  

# 調用 pdf_to_chunk 函數，將一個 PDF 文件的內容分割為多個文本塊
lst_chunk = pdf_to_chunk('TPASS.pdf')  # 替換 'your_file.pdf' 為實際的 PDF 文件路徑

# 打印第一個文本塊以驗證結果
print(lst_chunk[0])  

1 中部地區公共運輸定期票 售票與使用須知  
2023年6月15日起適用  
壹、  發行依據  
依據「交通部公路總局執行公共運輸通勤月票補助作業要點」，由 臺
中市政府、彰化縣政府、南投縣政府及苗栗縣政府 (以下簡稱地方政府 )委託
臺中捷運股份有限公司 (以下簡稱台中捷運公司 )發行「臺中市定期票」 、 「彰
化縣定期票」、「南投縣定期票」及「中彰投苗定期票」 。 
貳、  發售票種及說明  
中部地區 公共運輸 各方案定期票售價及可搭乘之運具範圍如表一及表
二，持卡人 於定期票有效期間內，不限里程、不限次數搭乘 定期票所屬之
公共運輸運具，適用範圍如下。  
表一 定期票售票方案1 
區域方案  購買對象  售價 (元) 可使用運具  
臺中市  
定期票  臺中市民  299 臺中市境內臺鐵、捷運 、市
區公車、公共自行車  非臺中市民  599 
彰化縣  
定期票  民眾  699 彰化縣境內之臺鐵2、公路客
運、市區公車、公共自行車  
南投縣  
定期票  民眾  699 南投縣境內之臺鐵2(集集
線)、公路客運、市區公車  
中彰投苗  
定期票  臺中市民  699 中彰投苗4縣市境內之臺
鐵、捷運、市區公車 、公共
自行車、公路客運  非臺中市民  999 
註1：各票種之適用範圍及路線以中彰投苗 4縣市政府及臺中捷運公司之公告為準。  
註2：購買彰化縣、南投縣定期票，初期尚無法使用臺鐵，俟完成整合後方可使用，並
將另行公告。


In [3]:
from sentence_transformers import SentenceTransformer  

# 將文本塊轉換為向量表示
def chunk_to_vector(chunks): 
    # 初始化使用的預訓練模型
    model = SentenceTransformer('all-MiniLM-L12-v2') 

    # 將文本塊列表轉換為向量表示
    arr_vector = model.encode(chunks) 
    
    # 返回向量列表
    return arr_vector 

# 調用 chunk_to_vector 函數，將文本塊列表轉換為向量表示
# arr_vectors 的形狀為 (n, 384)，其中 n 是文本塊的數量，384 是向量的維度
arr_vectors = chunk_to_vector(lst_chunk) 

# 打印向量數組的形狀以驗證結果
print(arr_vectors.shape)  # 例如輸出: (57, 384)


d:\Anaconda\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
d:\Anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


(9, 384)


In [4]:
# from qdrant_client import QdrantClient 
# from qdrant_client.http import models 
# from qdrant_client.http.models import PointStruct  

# # 連接到 Qdrant 並創建一個集合
# def connection(v_dim, collection): 
#     # v_dim: 向量維度, collection: 類似於資料庫中的表名稱

#     # 創建一個 Qdrant 客戶端，連接到本地 Qdrant 伺服器
#     client = QdrantClient("http://localhost:6333")  
    
#     # 重新創建一個集合（如果存在則刪除並重新創建）
#     client.recreate_collection(         
#         collection_name=collection,  # 集合名稱，相當於資料庫中的表名稱
#         vectors_config=models.VectorParams(  
#             distance=models.Distance.COSINE,  # 使用余弦距離作為向量相似度度量
#             size=v_dim  # 向量的維度，例如 384
#         ),         
#         optimizers_config=models.OptimizersConfigDiff(memmap_threshold=20000),  # 優化器配置，用於優化存儲和查詢性能
#         hnsw_config=models.HnswConfigDiff(on_disk=True, m=16, ef_construct=100)  # HNSW 配置，用於加速向量搜索
#     )     

#     # 返回 Qdrant 客戶端對象，用於後續的操作
#     return client  

# # 將向量及其對應的數據插入到 Qdrant 集合中
# def upsert_vector(client, collection, vectors, data): 
#     # vectors: 向量數組 (Array)
#     # data: 與向量對應的數據列表 (List of dictionaries)，如 [{"text": "example text"}]

#     # 遍歷所有的向量及其對應的數據
#     for i, vector in enumerate(vectors):         
#         client.upsert(  # 插入或更新向量和數據             
#             collection_name=collection,  # 指定要插入的集合
#             points=[PointStruct(  # 定義要插入的數據點
#                 id=i,  # 每個數據點的唯一標識符，這裡使用索引值
#                 vector=vectors[i],  # 要插入的向量
#                 payload=data[i]  # 與該向量相關聯的數據（例如文本）
#             )]         
#         )  

# # 准備將文本塊列表（chunks）轉換為包含文本的字典列表
# lst_dic_chunk = [] 
# for chunk in lst_chunk:     
#     lst_dic_chunk.append({"text": chunk})  # 將每個文本塊轉換為字典，並添加到列表中

# # 連接到 Qdrant 伺服器並創建/重新創建集合
# qclient = connection(v_dim=384, collection='重點') 

# # 將向量和對應的文本數據插入到 Qdrant 集合中
# upsert_vector(
#     client=qclient, 
#     collection='重點',  # 指定集合名稱
#     vectors=arr_vectors,  # 已經計算好的向量數組
#     data=lst_dic_chunk  # 對應的文本數據列表
# )


In [7]:
pip install faiss-cpu 


  Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl.metadata (3.8 kB)
Using cached faiss_cpu-1.8.0.post1-cp312-cp312-win_amd64.whl (14.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 加载 SentenceTransformer 模型
model = SentenceTransformer('all-MiniLM-L12-v2')

# 
question = '台中捷運公司發行哪四個定期票?'
question_emb = model.encode(question)

# 創建索引（余弦相似度使用 L2 歸一化）
index = faiss.IndexFlatL2(384)
faiss.normalize_L2(np.array([question_emb]))

# 插入向量
index.add(np.array([question_emb]))

# 查找向量
D, I = index.search(np.array([question_emb]), k=3)
print(I, D)  # I 是最近鄰的索引，D 是距離


[[ 0 -1 -1]] [[0.0000000e+00 3.4028235e+38 3.4028235e+38]]


In [10]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 初始化 SentenceTransformer 模型，用於將文本轉換為向量
model = SentenceTransformer('all-MiniLM-L12-v2')

# 定義要查詢的問題文本
question = '台中捷運公司發行哪四個定期票'

# 將問題文本編碼為向量表示
question_emb = model.encode(question)

# 假設你已經有一個存儲向量的矩陣 (例如來自多個文本塊)
# 這裡我們用 np.array 創建一個簡單的示例數據集
# 如果你有實際的數據集，將這個矩陣替換為實際的向量集
stored_vectors = np.random.rand(100, 384).astype('float32')  # 示例: 100 個 384 維向量

# 創建 FAISS 索引
index = faiss.IndexFlatL2(384)  # 384 是向量的維度，L2 是使用的距離度量

# 向索引中添加向量
index.add(stored_vectors)

# 執行向量搜索
# - query_vector: 用於查詢的向量 (question_emb)
# - k: 返回的最近鄰數量
D, I = index.search(np.array([question_emb], dtype='float32'), k=3)

# 打印搜索結果
print("最近鄰索引:", I)
print("距離:", D)


最近鄰索引: [[26 78 34]]
距離: [[117.33954 118.67166 120.04205]]


### 檢索答案:行「臺中市定期票」、「彰
### 化縣定期票」、「南投縣定期票」及「中彰投苗定期票」

In [33]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader 
from langchain.text_splitter import CharacterTextSplitter 
import re  

# 將 PDF 文件內容分割成多個文本塊的函數
def pdf_to_chunk(path, start_page=1, end_page=None): 
    loader = PyPDFLoader(path) 
    pages = loader.load() 
    total_pages = len(pages) 
    
    text_splitter = CharacterTextSplitter( 
        separator="\n",  # 使用 "\n" 作為分隔符
        chunk_size=256,  # 每個塊的最大字符數為 256
        chunk_overlap=20  # 塊之間有 20 個字符的重疊
    ) 

    if end_page is None: 
        end_page = len(pages) 

    lst_text = [] 
    
    for i in range(start_page-1, end_page): 
        chunks = text_splitter.split_text(pages[i].page_content) 
        for chunk in chunks: 
            text = re.sub(r'\s+', ' ', chunk) 
            lst_text.append(text) 
    
    return lst_text  

# Step 1: 分割 PDF 並生成文本塊
lst_chunk = pdf_to_chunk('TPASS.pdf')

# Step 2: 初始化 SentenceTransformer 模型，用於將文本塊轉換為向量
model = SentenceTransformer('all-MiniLM-L12-v2')

# 將文本塊轉換為向量
arr_vectors = np.array([model.encode(chunk) for chunk in lst_chunk])

# Step 3: 創建 FAISS 索引
index = faiss.IndexFlatL2(384)  # 384 是向量的維度，L2 是使用的距離度量

# 向索引中添加向量
index.add(arr_vectors)

# Step 4: 查詢向量資料庫並與 LLM 結合
# 定義要查詢的問題文本
question = '台中捷運公司發行哪四個定期票?'
question_emb = model.encode(question)

# 執行向量搜索，找到最相關的文本
D, I = index.search(np.array([question_emb]), k=1)  # 找到最相似的1個文本

# 根據檢索結果找到對應的文本
retrieved_text = lst_chunk[I[0][0]]

# 初始化 OpenAI 客戶端
client = OpenAI()

# 將檢索到的文本與問題一起傳遞給 LLM
completion = client.chat.completions.create(   
    model="gpt-4-turbo",   
    messages=[
        {"role": "system", "content": "你是一位聰明的 AI 助理，使用檢索到的背景資訊回答使用者問題。"},
        {"role": "system", "content": f"檢索到的背景資訊：{retrieved_text}"},
        {"role": "user", "content": question}
    ]
)

# 獲取 LLM 的回答
answer = completion.choices[0].message.content
print(answer)


台中捷運公司發行的四個定期票分別為「臺中市定期票」、「彰化縣定期票」、「南投縣定期票」以及「中彰投苗定期票」。
